In [46]:
import numpy as np
import pandas as pd
import os

In [26]:
def gap(row):
    if abs(row['dualBound']) > 999999999 or abs(row['primalBound']) > 999999999 or \
            row['dualBound'] * row['primalBound'] < 0:
        return 1
    else:
        num = abs(row['primalBound'] - row['dualBound'])
        den = max(abs(row['primalBound']), abs(row['dualBound']))
        return num/den

In [57]:
def infeasible(row):
    if abs(row['primalBound']) > 999999999:
        return 1
    else:
        return 0

In [133]:
def make_stats(directory):
    dfs = {}
    stats = []
    more_stats = []
    stems = []
    for csv in os.listdir(directory):
        stem = csv.split('.')[0]
        stems.append(stem)
        df = pd.read_csv(os.path.join(directory, csv))
        df['primalBound'] = pd.to_numeric(df['primalBound'], errors='coerce')
        df['heuristicPrimalBound'] = pd.to_numeric(df['heuristicPrimalBound'], errors='coerce')
        df['reltime'] = df['completionTime'] / df['maxCompletionTime']
        df['gap'] = df.apply(lambda row: gap(row), axis=1)
        df['nofeas'] = df.apply(lambda row: infeasible(row), axis=1)
        df['batch'] = np.array(df.index)//10 + 1
        stats.append(df[['reltime', 'gap', 'nofeas']].mean())
        dff = df.groupby('batch').mean()[['reltime', 'gap', 'nofeas']].reset_index()
        dff['instance set'] = stem
        more_stats.append(dff)
        dfs[stem] = df
    df = pd.DataFrame(stats)
    df['instance set'] = stems
    df = df[['instance set', 'reltime', 'gap', 'nofeas']]
    
    dff = pd.concat(more_stats)
    dff = dff[['instance set', 'batch', 'reltime', 'gap', 'nofeas']]
    
    return df, dfs, dff

In [134]:
baseline_df, baseline_dfs, baseline_dffs = make_stats('baseline')
vpc_df, vpc_dfs, vpc_dffs = make_stats('vpcs')

In [142]:
baseline_df.to_csv('baseline_average.csv', index=False, float_format='%.3f')
baseline_dffs.to_csv('baseline_batch_average.csv', index=False, float_format='%.3f')
vpc_df.to_csv('vpc_average.csv', index=False, float_format='%.3f')
vpc_dffs.to_csv('vpc_batch_average.csv', index=False, float_format='%.3f')

In [135]:
baseline_df

,instance set,reltime,gap,nofeas
0,rhs_series_1,1.004950,0.053974,0.000000
1,rhs_series_2,3.072333,0.005101,0.000000
2,matrix_series_1,1.023867,0.786404,0.000000
3,mat_rhs_bnd_obj_series_1,1.009222,0.161656,0.080000
4,obj_series_1,1.000550,0.006334,0.000000
5,rhs_obj_series_1,1.190333,0.218890,0.194444
6,obj_series_2,1.025867,0.970676,0.780000
7,bnd_series_2,1.004000,1.000000,1.000000


In [137]:
vpc_df

,instance set,reltime,gap,nofeas
0,rhs_series_1,1.006000,0.121353,0.0
1,rhs_series_2,3.835000,0.005232,0.0
2,matrix_series_1,1.023000,0.784001,0.0
3,mat_rhs_bnd_obj_series_1,1.023889,0.204857,0.1
4,obj_series_1,1.001000,0.006997,0.0
5,rhs_obj_series_1,4.336667,1.000000,1.0
6,obj_series_2,1.026000,1.000000,0.9
7,bnd_series_2,1.006667,1.000000,1.0
8,bnd_series_1,1.002167,0.431339,0.0


In [138]:
baseline_dffs

,instance set,batch,reltime,gap,nofeas
0,rhs_series_1,1,1.005000,0.048140,0.000000
1,rhs_series_1,2,1.005000,0.057582,0.000000
2,rhs_series_1,3,1.004750,0.047962,0.000000
3,rhs_series_1,4,1.005250,0.071238,0.000000
4,rhs_series_1,5,1.004750,0.044947,0.000000
0,rhs_series_2,1,3.191667,0.005232,0.000000
1,rhs_series_2,2,3.150000,0.005042,0.000000
2,rhs_series_2,3,3.233333,0.004999,0.000000
3,rhs_series_2,4,2.791667,0.005039,0.000000
4,rhs_series_2,5,2.995000,0.005193,0.000000


In [140]:
vpc_dffs

,instance set,batch,reltime,gap,nofeas
0,rhs_series_1,1,1.006000,0.121353,0.0
0,rhs_series_2,1,3.835000,0.005232,0.0
0,matrix_series_1,1,1.023000,0.784001,0.0
0,mat_rhs_bnd_obj_series_1,1,1.023889,0.204857,0.1
0,obj_series_1,1,1.001000,0.006997,0.0
0,rhs_obj_series_1,1,4.336667,1.000000,1.0
0,obj_series_2,1,1.026000,1.000000,0.9
0,bnd_series_2,1,1.006667,1.000000,1.0
0,bnd_series_1,1,1.002167,0.431339,0.0
